In [1]:
class Space:
    def __init__(self, *args):
        self.code = args[0]
        self.size = args[1]


class Group:
    def __init__(self, *args):
        self.id = args[0]
        self.size = args[1]


class Activity:
    def __init__(self, id, *args):
        self.id = id
        self.subject = args[0]
        self.teacher_id = args[1]
        self.group_ids = args[2]
        self.duration = args[3]

    def __repr__(self):
        return f"Activity(id={self.id}, subject={self.subject}, teacher_id={self.teacher_id}, group_ids={self.group_ids}, duration={self.duration})"


class Period:
    def __init__(self, *args):
        self.space = args[0]
        self.group = args[1]
        self.activity = args[2]

In [2]:
import json

# Load data from JSON file
with open('sliit_computing_dataset.json', 'r') as file:
    data = json.load(file)

# Create dictionaries to store instances
spaces_dict = {}
groups_dict = {}
activities_dict = {}
slots = []
# Populate the dictionaries with data from the JSON file
for space in data['spaces']:
    spaces_dict[space['code']] = Space(space['code'], space['capacity'])

for group in data['years']:
    groups_dict[group['id']] = Group(group['id'], group['size'])

for activity in data['activities']:
    activities_dict[activity['code']] = Activity(
        activity['code'], activity['subject'], activity['teacher_ids'][0], activity['subgroup_ids'], activity['duration'])

for day in ["MON", "TUE", "WED", "THU", "FRI"]:
    for id in range(1, 9):
        slots.append(day+str(id))
# Print the dictionaries to verify
print(spaces_dict)
print(groups_dict)
print(activities_dict)
print(slots)

{'LH401': <__main__.Space object at 0x000001B91AA20500>, 'LH501': <__main__.Space object at 0x000001B91A9EE450>, 'LAB501': <__main__.Space object at 0x000001B91A9EE840>, 'LAB502': <__main__.Space object at 0x000001B91AA9A570>}
{'Y1S1.1': <__main__.Group object at 0x000001B91AA9A5D0>, 'Y1S1.2': <__main__.Group object at 0x000001B91956FFB0>, 'Y1S1.3': <__main__.Group object at 0x000001B91AA9A5A0>, 'Y1S1.4': <__main__.Group object at 0x000001B91AA9A600>, 'Y1S1.5': <__main__.Group object at 0x000001B91AA9A630>, 'Y1S2.1': <__main__.Group object at 0x000001B91AA9A660>, 'Y1S2.2': <__main__.Group object at 0x000001B91AA9A690>, 'Y1S2.3': <__main__.Group object at 0x000001B91AA9A6C0>, 'Y1S2.4': <__main__.Group object at 0x000001B91AA9A6F0>, 'Y1S2.5': <__main__.Group object at 0x000001B91AA9A720>, 'Y2S1.1': <__main__.Group object at 0x000001B91AA9A750>, 'Y2S1.2': <__main__.Group object at 0x000001B91AA9A780>, 'Y2S1.3': <__main__.Group object at 0x000001B91AA9A7B0>, 'Y2S1.4': <__main__.Group objec

In [3]:
vacant_rooms = []


def evaluator(timetable):
    vacant_room = 0
    prof_conflicts = 0
    room_size_conflicts = 0
    sub_group_conflicts = 0
    unasigned_activities = len(activities_dict)
    activities_set = set()

    for slot in timetable:
        prof_set = set()
        sub_group_set = set()
        for room in timetable[slot]:
            activity = timetable[slot][room]

            if not isinstance(activity, Activity):
                vacant_room += 1
                vacant_rooms.append((slot, room))

            else:
                activities_set.add(activity.id)
                if activity.teacher_id in prof_set:
                    prof_conflicts += 1

                sub_group_conflicts += len(
                    set(activity.group_ids).intersection(sub_group_set))

                group_size = 0
                for group_id in activity.group_ids:
                    group_size += groups_dict[group_id].size
                    sub_group_set.add(group_id)

                if group_size > spaces_dict[room].size:
                    room_size_conflicts += 1
                teacher_id = activity.teacher_id
                prof_set.add(teacher_id)
    unasigned_activities -= len(activities_set)
    return vacant_room, prof_conflicts, room_size_conflicts, sub_group_conflicts, unasigned_activities
    # return f" Vacant Rooms : {vacant_room} \nLecturer Conflicts : {prof_conflicts}\nRoom Size Conflicts : {room_size_conflicts}\nSub Group Conflicts:{sub_group_conflicts} \nNo:of Unassigned Activities:{unasigned_activities} "

In [13]:
import random

POPULATION_SIZE = 50
NUM_GENERATIONS = 100
MUTATION_RATE = 0.1
CROSSOVER_RATE = 0.8


def get_classsize(activity: Activity) -> int:
    classsize = 0
    for id in activity.group_ids:
        classsize += groups_dict[id].size
    return classsize

def evaluate_population(population):
    """Evaluate each individual using the provided evaluator function."""
    fitness_values = []
    for timetable in population:
        fitness_values.append(evaluator(timetable))
    return fitness_values

def mutate(individual):
    """Perform mutation by randomly swapping activities in the timetable."""
    slots = list(individual.keys())
    slot1, slot2 = random.sample(slots, 2)
    room1, room2 = random.choice(list(individual[slot1])), random.choice(list(individual[slot2]))
    
    individual[slot1][room1], individual[slot2][room2] = individual[slot2][room2], individual[slot1][room1]

def crossover(parent1, parent2):
    """Perform crossover by swapping time slots between two parents."""
    child1, child2 = parent1.copy(), parent2.copy()
    slots = list(parent1.keys())
    split = random.randint(0, len(slots) - 1)
    
    for i in range(split, len(slots)):
        child1[slots[i]], child2[slots[i]] = parent2[slots[i]], parent1[slots[i]]
    
    return child1, child2

def fast_nondominated_sort(fitness_values):
    """Perform non-dominated sorting based on the multi-objective fitness values."""
    fronts = [[]]
    S = [[] for _ in range(len(fitness_values))]
    n = [0] * len(fitness_values)
    rank = [0] * len(fitness_values)

    for p in range(len(fitness_values)):
        for q in range(len(fitness_values)):
            if dominates(fitness_values[p], fitness_values[q]):
                S[p].append(q)
            elif dominates(fitness_values[q], fitness_values[p]):
                n[p] += 1
        if n[p] == 0:
            rank[p] = 0
            fronts[0].append(p)

    i = 0
    while fronts[i]:
        next_front = []
        for p in fronts[i]:
            for q in S[p]:
                n[q] -= 1
                if n[q] == 0:
                    rank[q] = i + 1
                    next_front.append(q)
        i += 1
        fronts.append(next_front)

    return fronts[:-1]

def dominates(fitness1, fitness2):
    """Return True if fitness1 dominates fitness2."""
    return all(f1 <= f2 for f1, f2 in zip(fitness1, fitness2)) and any(f1 < f2 for f1, f2 in zip(fitness1, fitness2))

def calculate_crowding_distance(front, fitness_values):
    """Calculate crowding distance for a front."""
    distances = [0] * len(front)
    num_objectives = len(fitness_values[0])

    for m in range(num_objectives):
        front.sort(key=lambda x: fitness_values[x][m])
        distances[0] = distances[-1] = float('inf')

        min_value = fitness_values[front[0]][m]
        max_value = fitness_values[front[-1]][m]
        if max_value == min_value:
            continue

        for i in range(1, len(front) - 1):
            distances[i] += (fitness_values[front[i + 1]][m] - fitness_values[front[i - 1]][m]) / (max_value - min_value)

    return distances

def select_parents(population, fitness_values):
    """Perform tournament selection based on non-dominated sorting and crowding distance."""
    fronts = fast_nondominated_sort(fitness_values)
    selected = []
    
    for front in fronts:
        if len(selected) + len(front) > POPULATION_SIZE:
            crowding_distances = calculate_crowding_distance(front, fitness_values)
            sorted_front = sorted(zip(front, crowding_distances), key=lambda x: x[1], reverse=True)
            selected.extend([x[0] for x in sorted_front[:POPULATION_SIZE - len(selected)]])
            break
        else:
            selected.extend(front)

    return [population[i] for i in selected]

def generate_initial_population():
    """Generate an initial population with random timetables."""
    population = []

    for _ in range(POPULATION_SIZE):
        timetable = {}
        activity_slots = {activity.id: []
                          for activity in activities_dict.values()}
        activities_remain = [activity.id for activity in activities_dict.values()
                             for _ in range(activity.duration)]
        for slot in slots:
            timetable[slot] = {}
            for space_id in spaces_dict.keys():
                space = spaces_dict[space_id]
                a_activities = [activity for activity in activities_remain if get_classsize(
                    activities_dict[activity]) <= space.size]
                a_activities = [
                    activity for activity in a_activities if slot not in activity_slots[activity]]
                activity = random.sample(a_activities,1)[0]
                timetable[slot][space_id] = activities_dict[activity]
                activity_slots[activity].append(slot)

        population.append(timetable)
    return population

def nsga2():
    """Main NSGA-II algorithm loop."""
    population = generate_initial_population()

    for generation in range(NUM_GENERATIONS):
        fitness_values = evaluate_population(population)
        new_population = []

        while len(new_population) < POPULATION_SIZE:
            parent1, parent2 = random.sample(population, 2)
            if random.random() < CROSSOVER_RATE:
                child1, child2 = crossover(parent1, parent2)
            else:
                child1, child2 = parent1.copy(), parent2.copy()

            if random.random() < MUTATION_RATE:
                mutate(child1)
            if random.random() < MUTATION_RATE:
                mutate(child2)

            new_population.extend([child1, child2])

        population = select_parents(new_population, evaluate_population(new_population))
    
    return population

# Run NSGA-II
final_population = nsga2()

In [14]:
final_population

[{'MON1': {'LH401': Activity(id=AC-140, subject=IT3550, teacher_id=FA0000007, group_ids=['Y3S2.4'], duration=1),
   'LH501': Activity(id=AC-139, subject=IT3550, teacher_id=FA0000001, group_ids=['Y3S2.3'], duration=1),
   'LAB501': Activity(id=AC-006, subject=IT1010, teacher_id=FA0000001, group_ids=['Y1S1.5'], duration=1),
   'LAB502': Activity(id=AC-138, subject=IT3550, teacher_id=FA0000010, group_ids=['Y3S2.2'], duration=1)},
  'MON2': {'LH401': Activity(id=AC-077, subject=IT2040, teacher_id=FA0000006, group_ids=['Y2S1.1', 'Y2S1.2', 'Y2S1.3', 'Y2S1.4', 'Y2S1.5'], duration=2),
   'LH501': Activity(id=AC-132, subject=IT3040, teacher_id=FA0000005, group_ids=['Y3S1.2'], duration=1),
   'LAB501': Activity(id=AC-189, subject=IT4560, teacher_id=FA0000006, group_ids=['Y4S2.5'], duration=1),
   'LAB502': Activity(id=AC-189, subject=IT4560, teacher_id=FA0000006, group_ids=['Y4S2.5'], duration=1)},
  'MON3': {'LH401': Activity(id=AC-189, subject=IT4560, teacher_id=FA0000006, group_ids=['Y4S2.5']

In [ ]:
def adapter(solution):
    timetable = {}
    for slot in slots:
        timetable[slot] = {}
        for space in spaces_dict.keys():
            timetable[slot][space] = ""

    for period in solution:
        time_slots = period[3]
        room = period[2]
        activity_id = period[0]
        if not isinstance(time_slots, list):
            time_slots = [time_slots]
        for aslot in time_slots:
            timetable[aslot][room] = activities_dict[activity_id]
    return timetable


def get_classsize(activity: Activity) -> int:
    classsize = 0
    for id in activity.group_ids:
        classsize += groups_dict[id].size
    return classsize

def create_timetable():
    population = []
    
    for _ in range(POPULATION_SIZE):
        timetable = {}
        activity_slots = {activity.id :[] for activity in activities_dict.values}
        activities_remain = [activity.id for activity in activities_dict.values()
                             for _ in range(activity.duration)]
        for slot in slots:
            timetable[slot] = {}
            for space_id in spaces_dict.keys():
                space = spaces_dict[space_id]
                a_activities = [activity for activity in activities_remain if get_classsize(activities_dict[activity]) <=space.size]
                a_activities = [activity for activity in a_activities if slot not in activity_slots[activity]]
                activity = random.sample(a_activities)
                timetable[slot][space] = activity
                activity_slots[activity].append(slot)

        population.append(timetable)
    return population
                # timetable[slot][space] 
        

In [15]:
evaluator(final_population[0])

(0, 28, 1, 10, 79)

In [17]:
def print_metric(vacant_rooms, prof_conflicts, room_size_conflicts, sub_group_conflicts, unassigned_activities):
    a = f" Vacant Rooms : {vacant_rooms} \nLecturer Conflicts : {prof_conflicts}\nRoom Size Conflicts : {room_size_conflicts}\nSub Group Conflicts:{sub_group_conflicts} \nNo:of Unassigned Activities:{unassigned_activities} "
    print(a)

print_metric(*evaluator(final_population[0]))

 Vacant Rooms : 0 
Lecturer Conflicts : 28
Room Size Conflicts : 1
Sub Group Conflicts:10 
No:of Unassigned Activities:79 
